In [4]:
!pip3 install scikit-learn

     ---------------------------------------- 7.1/7.1 MB 41.5 MB/s eta 0:00:00
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [21]:
!pip3 install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [6]:
!pip3 install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)


In [7]:
# -*- coding: utf-8 -*-
import chatterbot.corpus
from chatterbot import comparisons
from chatterbot import response_selection
from chatterbot import ChatBot
from chatterbot.comparisons import LevenshteinDistance
from chatterbot.response_selection import get_first_response
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot.trainers import ListTrainer
import pandas as pd
import numpy as np
import random
import string
import sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import glob
import os
import nltk
import logging
logging.basicConfig(level=logging.INFO)

# first-time use only
# nltk.download('punkt') 

# first-time use only
# nltk.download('wordnet')

In [104]:
def faq_chatbot_initialize(chatbot_name, threshold=0.9, excel_path='data\testforkb.xlsx', worksheet_name='FAQ'):
    covid_faq_chatbot = ChatBot(
        chatbot_name,
        logic_adapters=[
            {
                "import_path": "chatterbot.logic.BestMatch",
                "statement_comparison_function": LevenshteinDistance,
                "response_selection_method": get_first_response,
                "maximum_similarity_threshold": threshold
            }
        ],
        preprocessors=[
            'chatterbot.preprocessors.clean_whitespace'
        ],
        read_only=True,
    )
    trainer = ListTrainer(covid_faq_chatbot)
    #trainer.train("chatterbot.corpus.english")
    # read questions and answers
    data = pd.read_excel(excel_path, sheet_name=worksheet_name, engine='openpyxl')
    question = data.get('Queston')
    answer = data.get('Long_Answer')

    #for i in range(0, 3):
    #    print('[Q]', question[i], '\n[A]', answer[i], '\n\n')

    # Iteratively adding the question and answer
    train_list = []
    for i in range(len(question)):
        train_list.append(question[i])
        train_list.append(answer[i])
    # train the faq
    trainer.train(train_list)
    trainer.export_for_training('covid.yml')
    return covid_faq_chatbot




In [105]:
class NLP_Chatbot:
    
    def __init__(self, name, file_path):
        self.name = name
        self.file_path = file_path
        self.sents = self.generate_sents()
        self.TfidfVec, self.tfidf = self.generate_tfidf()

    def generate_sents(self):
        raw = []
        for filename in glob.glob(os.path.join(self.file_path, '*.txt')):
            with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
                # do your stuff
                lines = f.readlines()
            raw.extend(lines)
        sents = [ nltk.sent_tokenize(r) for r in raw ] # converts each paragraph to a list of sentences 
        sents = [ s for sent in sents for s in sent  ] # flatten the list
        return sents
    
    def generate_tfidf(self):

        # prepare for lemmatization
        WNL = nltk.stem.WordNetLemmatizer()
        #for handling some known bugs while not using POS tag info.
        exceptions = ['has', 'was', 'as', 'us', 'less']
        def MyNormalize(text):
            tokens=nltk.word_tokenize(text.lower())
            tokens=[ t for t in tokens if t not in string.punctuation ]
            toks = [WNL.lemmatize(t) if t not in exceptions else t for t in tokens  ]
            return toks
        # Prepare a preprocessing function that will do tokenization,
        # case lowering, punctuation removal, and lemmatization
        my_stop_words = text.ENGLISH_STOP_WORDS

        # preprocess the sentences in data, remove stop words, and create a tf-idf vector
        TfidfVec = TfidfVectorizer(tokenizer=MyNormalize, stop_words=my_stop_words)
        tfidf = TfidfVec.fit_transform(self.sents)
        return TfidfVec, tfidf
    
    # function to match input to the preprocessed sentences
    def get_response(self, user_response):
        robo_response=''
        new = self.TfidfVec.transform([user_response])
        vals = cosine_similarity(new[0], self.tfidf)
        idx=vals.argsort()[0][-1]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-1]
        if(req_tfidf==0):
            robo_response=robo_response+"I am sorry! I don't understand you."
            return robo_response
        else:
            robo_response = robo_response+self.sents[idx]
            return robo_response    
    



In [106]:
def nlp_chatbot_initialize(chatbot_name,file_path='data/'):
    chatbot = NLP_Chatbot(chatbot_name, file_path)
    return chatbot


In [113]:
def get_answer(faq_chatbot, nlp_chatbot, question, threshold):  # let's get a response to our input
    # try suggested corpora to find best fit. If first corpus < theshold, try another.
    # avoid random responses confidence 0
    response = faq_chatbot.get_response(question)
    if  response.confidence < threshold:  # not a good answer, look elsewhere
        #response = nlp_chatbot.get_response(question)
        response = '123'
    else:
        response = response.serialize()['text']
    return response
#import logginglogging.basicConfig(level=logging.INFO)    # Enable info level logging


In [65]:
data = pd.read_excel('C:\\Users\\sanje\\data\\testforkb.xlsx', sheet_name='FAQ', engine='openpyxl')

In [62]:
data.to_csv('df_output.csv', encoding='utf-8')

In [68]:

# Define a function to remove non-UTF characters
def remove_non_utf(text):
    return re.sub(r'[^\x00-\x7F]+', '', text)

# Apply the function to the entire DataFrame
df_cleaned = data.applymap(remove_non_utf)

In [67]:
import re

In [69]:
df_cleaned.to_csv('df_output2.csv', encoding='utf-8')

In [168]:
data_path = 'data/'
excel_name =  'C:\\Users\\sanje\\data\\testforkb.xlsx'
worksheet_name = 'FAQ'


covid_faq_chatbot = faq_chatbot_initialize("Covid FAQ Chat Bot", excel_path=excel_name, worksheet_name=worksheet_name)
#covid_nlp_chatbot = nlp_chatbot_initialize("Covid NLP Chat Bot", data_path)



List Trainer: [####################] 100%


In [150]:
threshold=0.3
question = "fish"
#question = "what does Conversational AI refer to?"

print(get_answer(covid_faq_chatbot, covid_nlp_chatbot, question, threshold))

INFO:chatterbot.chatterbot:Beginning search for close text match
INFO:chatterbot.chatterbot:Processing search results
INFO:chatterbot.chatterbot:Similar text found: Shellfish 0.62
INFO:chatterbot.chatterbot:Using "Shellfish" as a close match to "fish" with a confidence of 0.62
INFO:chatterbot.chatterbot:Selecting response from 1 optimal responses.
INFO:chatterbot.response_selection:Selecting first response from list of 1 options.
INFO:chatterbot.chatterbot:Response selected. Using "Option 6"
INFO:chatterbot.chatterbot:BestMatch selected "Option 6" as a response with a confidence of 0.62


Option 6


In [153]:
#prepare some greeting words
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence): 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)






In [156]:
#starting the bot
flag=True
print("CHATTY: My name is CHATTY. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("CHATTY: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("CHATTY: "+greeting(user_response))
            else:
                print("CHATTY: ",end="")
                print(get_answer(covid_faq_chatbot, covid_nlp_chatbot, user_response, threshold))
    else:
        flag=False
        print("CHATTY: Bye! take care...")
        
# now chat with your bot...
# you may experiment with different similarity functions

CHATTY: My name is CHATTY. I will answer your queries about Chatbots. If you want to exit, type Bye!


KeyboardInterrupt: Interrupted by user

In [157]:
d = {'key':'value'}
d['faq'] = covid_faq_chatbot
d['nlp'] = covid_nlp_chatbot
print(d)

{'key': 'value', 'faq': <chatterbot.chatterbot.ChatBot object at 0x00000214D38B7848>, 'nlp': <__main__.NLP_Chatbot object at 0x0000021441864648>}


In [158]:
covid_faq_chatbot.trainer.export_for_training('./covid.yml')

AttributeError: 'ChatBot' object has no attribute 'trainer'

In [159]:
chatbot = ChatBot(
    'Covid FAQ Chat Bot',

    logic_adapters= [
            {
                "import_path": "chatterbot.logic.BestMatch",
                "statement_comparison_function": "chatterbot.comparisons.LevenshteinDistance",
                "response_selection_method": "chatterbot.response_selection.get_first_response",
                "maximum_similarity_threshold": 0.9
            }
    ],
    read_only=True,
    trainer= 'chatterbot.trainers.ListTrainer',
    training_data= [
         'chatterbot.corpus.custom.covid'
    ]
)


In [165]:

trainer = ChatterBotCorpusTrainer(chatbot)

# Train the chatbot based on the english corpus
trainer.train("covid")

TypeError: load() missing 1 required positional argument: 'Loader'

In [167]:
trainer.train('C:\\Users\\sanje\\covid.yml')
question = "fish"
chatbot.get_response(question)

TypeError: load() missing 1 required positional argument: 'Loader'